### Obtention des cultures pures

L'objectif de ce script est d'obtenir l'ensemble des cultures qui ne possèdent qu'un seul composant de culture. 
En travaillant avec les espèces pures, on a par la suite une bijection entre les cultures et les composants de culture, ce qui facilite la valorisation et le travail sur des cultures données.

In [2]:
import pandas as pd
from tqdm import tqdm
import json
ENTREPOT_PATH = '~/Bureau/utils/data/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'noeuds_synthetise', 
    'composant_culture', 
    'culture', 
    'noeuds_synthetise', 
    'espece'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)


100%|██████████| 5/5 [00:01<00:00,  3.67it/s]


In [3]:
# on ajoute à chaque composant culture son espece
left = df['composant_culture']
right = df['espece'][['nom_culture_acta']]
df['composant_culture_extanded'] = pd.merge(left, right, left_on='espece_id', right_index=True, how='left')

In [4]:
# on compte le nombre de composant dans chaque culture
culture_counts = df['composant_culture_extanded'].groupby('culture_id').size()

# les cultures sont définies comme pures si elles ne sont composées que d'un seul composants. 
is_pure = list(culture_counts[culture_counts == 1].index)

# on a toutes les cultures pures
df['culture_pure'] = df['culture'].loc[
    df['culture'].index.isin(is_pure)
]

In [5]:
# on ne conserve que les composant de cultures qui composent les espèces pures
df['composant_culture_pure'] = df['composant_culture_extanded'].loc[
    df['composant_culture_extanded']['culture_id'].isin(list(df['culture_pure'].index))
]

In [6]:
# maintenant qu'on est sur d'avoir une culture pure, on peut faire un left join avec composant_culture_extanded
left = df['culture_pure'][['code', 'nom']].rename(columns={'code' : 'culture_code', 'nom' : 'culture_nom'})
right = df['composant_culture_pure'][['culture_id', 'nom_culture_acta']]
result = pd.merge(left, right, left_index=True, right_on='culture_id', how='left')

### Exemple sur le poireau

In [7]:
# si on s'attarde sur les résultats pour le poireaux : 
result.loc[
    result['nom_culture_acta'].str.contains('poireau') == True
][['culture_id','culture_code', 'nom_culture_acta', 'culture_nom']]

,culture_id,culture_code,nom_culture_acta,culture_nom
id,,,,
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_1b447c62-2c23-46a5-8a99-a5e0f02571b2,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,98f102cf-2cae-4616-a991-c8dd1adaf674,poireau,Poireau
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_eef45c70-55c5-4ac4-a523-2b7fd8e5606c,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,a239ba3a-1aea-471f-975e-5a0023491dc3,poireau,Poireau
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_b17d11ce-0944-4355-821d-f45ee5507883,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,67c46405-a048-4296-8f96-e392219ac60f,poireau,Poireau d'hiver
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_3f691ede-f66b-47be-8319-4e240457da3a,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,c585e58b-f86c-4862-93cc-b34018b96d0e,poireau,poireaux
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_aa78bda3-8da2-4dee-a2a5-bc86da40086a,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,eea50975-7901-40ad-8e70-7ffadfea2a5c,poireau,Poireau
...,...,...,...,...
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_431a6dec-8681-4615-bb57-66379da278ab,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,bb8bc9aa-c109-4170-bf58-a4b2f75bf1b9,poireau,Poireau
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_cba0da1b-e070-476a-b384-8c0269100ddf,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,2ed80db6-65ee-805e-1336-c9ccdedb416b,poireau,poireau
fr.inra.agrosyst.api.entities.CroppingPlanSpecies_bc9419c4-fced-4d42-9cb8-eb7b27c40503,fr.inra.agrosyst.api.entities.CroppingPlanEntr...,2ed80db6-65ee-805e-1336-c9ccdedb416b,poireau,poireau
